<a href="https://colab.research.google.com/github/dadang6842/AI-study/blob/main/practice/recurrence_regression/Multi_Step_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Step LSTM Models
- 미래의 여러 시점을 예측, 출력층(Dense) 유닛 수 = M
- Single-Step은 한 시점만 예측, 출력층(Dense) 유닛 수 = 1

```
e.g.

Input
[10, 20, 30]

Output
[40, 50]
```


- 참고 자료: https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/


## Data Preparation
- split_sequence() 함수에서 n_steps_in, n_steps_out 인자를 받음

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

## Vector Output Model
- 벡터를 출력

In [ ]:
# univariate multi-step vector-output stacked lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# define model -> use a Stacked LSTM
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out)) # 출력 개수
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=50, verbose=0)

# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[[109.1352   123.342094]]


## Encoder-Decoder Model
- = sequence-to-sequence 모델
- 입력, 출력 시퀀스의 길이가 가변
- 모델이 두 개 -> encoder와 decoder

Encoder
- 입력 시퀀스를 읽어들여 하나의 고정 길이 벡터로 요약
- (batch_size, n_unit) 벡터를 출력

RepeatVector
- 출력 시퀀스의 길이만큼 인코더 출력을 반복
- 출력이 여러 타임스텝으로 구성되어야 하므로 같은 벡터를 여러 번 복제해서 (batch_size, n_steps_out, n_unit) 형태를 만듦

Decoder
- (batch_size, n_steps_out, n_unit) 벡터를 입력받아 LSTM으로 time step마다 출력 시퀀스를 생성 -> (batch_size, time_steps, n_unit)
- return_sequences=True → 전체 시퀀스를 출력하겠다는 뜻

TimeDistributed(Dense)
- 디코더가 출력하는 각 시점에 대해 Dense(1)를 적용하여 실제 예측값을 산출
- 최종 출력: (batch_size, n_steps_out, 1)
- n_steps_out개의 예측값이 1차원으로 나오는 구조

In [ ]:
# univariate multi-step encoder-decoder lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=100, verbose=0)

# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[[104.17813]
  [117.17174]]]


### Vector Output Model vs. Encoder-Decoder Model
- Encoder-Decoder는 추론 시 출력 길이 조절 가능 (auto-regressive(디코더의 출력을 다음 입력으로 반복) 사용 시)
- Encoder-Decoder는 출력 시퀀스 간 시간적 의존성을 잘 반영, 디코더(LSTM)에서 시점 간 연관성이 있기 때문
-  Vector Output은 한 번에 모든 예측값을 독립적으로 계산, 출력 시점 간 의존성 없음
